In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import math

In [5]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [0]:
cococapdf = pd.read_csv('/content/drive/My Drive/Colab_Data/captions.csv', names=['caption'])

In [8]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)


batch_size = 10000
n_batches = math.ceil(len(cococapdf) / batch_size)

start_from = 25 ## ONLY != 0 if crashed in the middle


for this_batch_n in range(start_from, n_batches):
    start_index = batch_size * this_batch_n
    end_index = batch_size * (this_batch_n + 1)
    print(this_batch_n, start_index, end_index)

    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        caption_embeddings = session.run(embed(cococapdf[start_index:end_index]['caption'].tolist()))
        np.savez_compressed('/content/drive/My Drive/Colab_Data/captions_{}.npz'.format(this_batch_n), caption_embeddings)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
25 250000 260000
26 260000 270000
27 270000 280000
28 280000 290000
29 290000 300000
30 300000 310000
31 310000 320000
32 320000 330000
33 330000 340000
34 340000 350000
35 350000 360000
36 360000 370000
37 370000 380000
38 380000 390000
39 390000 400000
40 400000 410000
41 410000 420000


In [0]:
parts = []
for n in range(42):
    parts.append(np.load('/content/drive/My Drive/Colab_Data/captions_{}.npz'.format(n))['arr_0'])

In [0]:
all_captions = np.concatenate(parts)

In [22]:
all_captions.shape

(414113, 512)

In [0]:
np.savez_compressed('/content/drive/My Drive/Colab_Data/captions.npz', all_captions)